workbook for migrating database: filemaker to sqlite

In [1]:
import sqlite3, os
import pandas as pd
import re

In [2]:
#set home directory path
hdir = os.path.expanduser('~')

dh_path = '/Dropbox/Active_Directories/Digital_Humanities/'

database_path = os.path.join(hdir, dh_path.strip('/'), 'database_eurasia_7.0.db')

In [3]:

# Check if database file exists
if not os.path.exists(database_path):
    raise FileNotFoundError(f"Database file not found at: {database_path}")

# Connect to the SQLite database
conn = sqlite3.connect(database_path)

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

try:
    # Your database operations will go here
    pass
finally:
    # Always close the connection when done
    cursor.close()
    conn.close()

In [3]:
# Connect to the SQLite database
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

try:
    # Query to get all table names
    cursor.execute("""
        SELECT name 
        FROM sqlite_master 
        WHERE type='table'
        ORDER BY name;
    """)
    
    # Fetch all results
    tables = cursor.fetchall()
    
    print("Tables in database:")
    for table in tables:
        table_name = table[0]
        # Query to get the number of columns and foreign keys for each table
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns_info = cursor.fetchall()
        num_columns = len(columns_info)

        cursor.execute(f"PRAGMA foreign_key_list({table_name});")
        foreign_keys = cursor.fetchall()
        foreign_keys_info = [fk[3] for fk in foreign_keys]  # Get the foreign key names

        print(f"- Table: {table_name}, Number of Columns: {num_columns}, Foreign Keys: {foreign_keys_info}")

finally:
    cursor.close()
    conn.close()

Tables in database:
- Table: bibliography, Number of Columns: 22, Foreign Keys: ['Repository_ID']
- Table: classical_genre, Number of Columns: 7, Foreign Keys: ['Source_ID', 'Knowledge_Form_ID', 'Classical_ID']
- Table: classical_sources, Number of Columns: 20, Foreign Keys: ['Location_ID']
- Table: commentaries, Number of Columns: 5, Foreign Keys: ['Commentating_Work', 'Commentated_Work']
- Table: conquests, Number of Columns: 10, Foreign Keys: ['Defending_Power_ID', 'Conquering_Power_ID', 'Conquered_Territory_ID']
- Table: copies_holdings, Number of Columns: 14, Foreign Keys: ['Scribe_Individual_ID', 'Repository_ID', 'Reference_Source_ID', 'Location_ID', 'Copied_Source_ID', 'Copied_Classical_ID']
- Table: definitions, Number of Columns: 10, Foreign Keys: ['Source_ID', 'Social_Role_ID', 'Lexicon_ID']
- Table: epochs, Number of Columns: 6, Foreign Keys: []
- Table: gazetteer, Number of Columns: 13, Foreign Keys: []
- Table: honorifics, Number of Columns: 4, Foreign Keys: []
- Table: in

### Read in XML data

In [6]:
from bs4 import BeautifulSoup

In [13]:
notes_path = '/Dropbox/Active_Directories/Notes/Primary_Sources/xml_notes_stage2/parser_depository'

xml_directory = os.path.join(hdir, notes_path.strip('/'))

In [16]:
xml_files = [f for f in os.listdir(xml_directory) if f.endswith('.xml')]

xml_files


['ser1270.xml',
 'ser2037.xml',
 'ser1258.xml',
 'ser2023.xml',
 'ser193.xml',
 'ser187.xml',
 'ser811.xml',
 'ser1927.xml',
 'ser1933.xml',
 'ser1890.xml',
 'ser1884.xml',
 'ser1489.xml',
 'ser596.xml',
 'ser1885.xml',
 'ser970.xml',
 'ser1891.xml',
 'ser958.xml',
 'ser179.xml',
 'ser1932.xml',
 'ser1926.xml',
 'ser1701.xml',
 'ser2022.xml',
 'ser2036.xml',
 'ser1271.xml',
 'ser1515.xml',
 'ser1501.xml',
 'ser1529.xml',
 'ser2020.xml',
 'ser2034.xml',
 'ser812.xml',
 'ser621.xml',
 'ser1918.xml',
 'ser1930.xml',
 'ser1924.xml',
 'ser972.xml',
 'ser1887.xml',
 'ser1893.xml',
 'ser1878.xml',
 'ser1879.xml',
 'ser1892.xml',
 'ser967.xml',
 'ser973.xml',
 'ser1886.xml',
 'ser1925.xml',
 'ser1931.xml',
 'ser1919.xml',
 'ser813.xml',
 'ser1702.xml',
 'ser2035.xml',
 'ser2021.xml',
 'ser1528.xml',
 'ser2009.xml',
 'ser1514.xml',
 'ser2025.xml',
 'ser2031.xml',
 'ser1538.xml',
 'ser2019.xml',
 'ser817.xml',
 'ser1935.xml',
 'ser1921.xml',
 'ser1882.xml',
 'ser963.xml',
 'ser1896.xml',
 'ser98

In [18]:
# Assuming xml_files is already defined and contains the list of XML file names
xml_texts = {}  # Initialize an empty dictionary to store the file contents

for xml_file in xml_files:
    file_path = os.path.join(xml_directory, xml_file)  # Construct the full file path
    with open(file_path, 'r', encoding='utf-8') as file:  # Open the XML file
        xml_content = file.read()  # Read the full text of the file
        xml_texts[xml_file] = xml_content  # Store the content in the dictionary

# Now xml_texts contains the file names as keys and their contents as values

In [20]:
#xml_texts['ser2036.xml']

In [21]:
import pandas as pd
import sqlite3

# Assuming xml_texts is your dictionary of XML texts
# Example: xml_texts = {'ser2036.xml': '...xml content...', 'ser2037.xml': '...xml content...'}

# Step 1: Create a DataFrame from the dictionary
df = pd.DataFrame(list(xml_texts.items()), columns=['File_Name', 'XML_Text'])

# Step 2: Extract UID from File_Name
df['UID'] = df['File_Name'].str.extract(r'ser(\d+)\.xml')[0].astype(int)

# Step 3: Connect to the SQLite database
hdir = os.path.expanduser('~')
dh_path = '/Dropbox/Active_Directories/Digital_Humanities/'
database_path = os.path.join(hdir, dh_path.strip('/'), 'database_eurasia_7.0.db')

conn = sqlite3.connect(database_path)

# Step 4: Query the bibliography table for Date_Pub_Greg
query = "SELECT UID, Date_Pub_Greg FROM bibliography WHERE UID IN ({})".format(','.join(map(str, df['UID'].unique())))
bibliography_data = pd.read_sql_query(query, conn)

# Step 5: Merge the DataFrame with the bibliography data
df = df.merge(bibliography_data, on='UID', how='left')

# Step 6: Clean up the DataFrame (optional)
df.drop(columns=['File_Name'], inplace=True)  # Drop the original file name if not needed

# Close the database connection
conn.close()

# Display the resulting DataFrame
print(df)

                                              XML_Text   UID  Date_Pub_Greg
0    <?xml-model href="../../../../../../Projects/x...  1270    1885.772687
1    <?xml-model href="../../../../../../Projects/x...  2037       0.000000
2    <?xml-model href="../../../../../../Projects/x...  1258    1884.802458
3    <?xml-model href="../../../../../../Projects/x...  2023    1919.000000
4    <?xml-model href="../../../../../Projects/xml_...   193    1886.000000
..                                                 ...   ...            ...
348  <?xml-model href="../../../../../Projects/xml_...   808            NaN
349  <?xml-model href="../../../../../../Projects/x...  1533    1898.385664
350  <?xml-model href="../../../../../../Projects/x...  1255    1883.832229
351  <?xml-model href="../../../../../../Projects/x...  1527    1917.790244
352  <?xml-model href="../../../../../../Projects/x...  2012       0.000000

[353 rows x 3 columns]


In [22]:
df

,XML_Text,UID,Date_Pub_Greg
0,"<?xml-model href=""../../../../../../Projects/x...",1270,1885.772687
1,"<?xml-model href=""../../../../../../Projects/x...",2037,0.000000
2,"<?xml-model href=""../../../../../../Projects/x...",1258,1884.802458
3,"<?xml-model href=""../../../../../../Projects/x...",2023,1919.000000
4,"<?xml-model href=""../../../../../Projects/xml_...",193,1886.000000
...,...,...,...
348,"<?xml-model href=""../../../../../Projects/xml_...",808,NaN
349,"<?xml-model href=""../../../../../../Projects/x...",1533,1898.385664
350,"<?xml-model href=""../../../../../../Projects/x...",1255,1883.832229
351,"<?xml-model href=""../../../../../../Projects/x...",1527,1917.790244
